In [1]:
import re
import pandas as pd
from profanity import profanity
import csv
import numpy as np
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
import nltk
import spacy
from textblob import TextBlob
nlp = spacy.load('de_core_news_sm')
RAW_DATA_PATH = 'data/raw/'

Helper Functions

In [2]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [3]:
def find_longest_word(word_list):
    longest_word =  max(word_list, key=len)
    return len(longest_word)

In [4]:
def lemmatize_string(input_text):
    doc = nlp(input_text.lower())
    result = ' '.join([x.lemma_ for x in doc])
    doc = nlp(result.title())
    result = ' '.join([x.lemma_ for x in doc]).upper()
    return result

Data Extraction

In [118]:
#df_Lied = pd.read_csv(RAW_DATA_PATH + 'LIED.csv', usecols=['ID','INTERPRET', 'TITEL', 'SPRACHE_DEUTSCH', 'TEXT_TEIL1', 'TEXT_TEIL2', 'TEXT_TEIL3', 'TEXT_TEIL4'])
#print(df_Lied.head())

df_Chart_Position = pd.read_csv(RAW_DATA_PATH + 'CHART_POSITION.csv', usecols=['LIED_ID', 'POSITION', 'DATUM_VON', 'DATUM_BIS'])
print(df_Chart_Position.head())

with open(RAW_DATA_PATH+'Stoppwords.csv', newline='', encoding='UTF-8') as f:
    stopwords_list = list(csv.reader(f))
stopwords_list = [word.upper() for word in flatten(stopwords_list)]

   LIED_ID  POSITION   DATUM_VON   DATUM_BIS
0    26024        26  1983-04-18  1983-04-24
1     1073        46  1984-06-04  1984-06-10
2     1131        33  1984-06-18  1984-06-24
3    15819        22  1984-08-13  1984-08-19
4     7324         6  1984-08-27  1984-09-02


In [48]:
print(df.head(30))

         ID INTERPRET    TITEL  \
0   1148163     Yasha  Strand    
1   1148163     Yasha  Strand    
2   1148163     Yasha  Strand    
3   1148163     Yasha  Strand    
4   1148163     Yasha  Strand    
5   1148163     Yasha  Strand    
6   1148163     Yasha  Strand    
7   1148163     Yasha  Strand    
8   1148163     Yasha  Strand    
9   1148163     Yasha  Strand    
10  1148163     Yasha  Strand    
11  1148163     Yasha  Strand    
12  1148163     Yasha  Strand    
13  1148163     Yasha  Strand    
14  1148163     Yasha  Strand    
15  1148163     Yasha  Strand    
16  1148163     Yasha  Strand    
17  1148163     Yasha  Strand    
18  1148163     Yasha  Strand    
19  1148163     Yasha  Strand    
20  1148163     Yasha  Strand    
21  1148163     Yasha  Strand    
22  1148163     Yasha  Strand    
23  1148163     Yasha  Strand    
24  1148163     Yasha  Strand    
25  1148163     Yasha  Strand    
26  1148163     Yasha  Strand    
27  1148163     Yasha  Strand    
28  1148163   

Data Conversion

In [119]:
#df_Lied['Text'] = df_Lied['TEXT_TEIL1'].fillna('') + df_Lied['TEXT_TEIL2'].fillna('') + df_Lied['TEXT_TEIL3'].fillna('') + df_Lied['TEXT_TEIL4'].fillna('')


df_Chart_Position['DATUM_VON'] = pd.to_datetime(df_Chart_Position['DATUM_VON'])
df_Chart_Position['DATUM_BIS'] = pd.to_datetime(df_Chart_Position['DATUM_BIS'])
df_Chart_Position['DAUER'] = (df_Chart_Position['DATUM_BIS'] - df_Chart_Position['DATUM_VON']).dt.days.astype('int16')
df_Chart_Position['Jahr'] = df_Chart_Position['DATUM_BIS'].dt.year.astype('int16')
df_Chart_Position['Monat'] =  df_Chart_Position['DATUM_BIS'].dt.month.astype('int16')

Text Preprocessing

In [7]:
df_Lied['processed_Text'] = df_Lied['Text']

# ngram

#lemmatization
df_Lied['processed_Text'] = df_Lied.processed_Text.apply(lambda text: lemmatize_string(text))

#Stemming???
#df_Lied['processed_Text'] = df_Lied.processed_Text.apply(lambda text: lemmatize_string(text))

# tokenize
df_Lied['processed_Text'] = df_Lied.processed_Text.apply(lambda text: nltk.word_tokenize(text))

#remove stopwords
df_Lied['processed_Text'] = df_Lied['processed_Text'].apply(lambda x: [item for item in x if item not in stopwords_list])

#remove numbers
df_Lied['processed_Text'] = df_Lied['processed_Text'].apply(lambda word_list : [re.sub('\w*\d\w*','', word) for word in word_list])

In [8]:
print(df_Lied['processed_Text'][0])

['LASS', 'STRAND', 'WEISSEN', 'BRAUCH', 'MAL', 'WIEDER', 'MEER', 'SAND', 'YEAH', 'BRAUCH', 'MAL', 'WIEDERGRUND', 'BLEIBEN', 'AI', 'UHR', 'FAELLT', 'BLAU', 'SEHEN', 'VERSINKEN', 'SETZEN', 'LANGSAM', 'WASSER', 'SPIEGELGLATT', 'BLINKEN', 'WOLLEN', 'SEHNSUCHT', 'MORGEN', 'HEUTE', 'PERFEKT', '--', 'SEHNSUCHT', 'MORGEN', 'HEUTE', 'PERFEKT', '--', 'STEHST', 'ZEIGST', 'ECHT', 'LICHTER', 'STADT', 'VERBRANNT', 'LASS', 'STRAND', 'KOMMN', 'STEIGEN', 'LEUCHTEST', 'DIAMANT', 'YEHE', 'ALL', 'SALZ', 'HAUT', 'AI', 'SEH', 'GESTOCHEN', 'SCHARF', 'TUN', 'FAST', 'AUGE', 'WEH', 'ENDLICH', 'NOCH', 'NIE', 'SEHEN', 'YEHI', 'WOLLEN', 'SEHNSUCHT', 'MORGEN', 'HEUTE', 'PERFEKT', '--', 'SEHNSUCHT', 'MORGEN', 'HEUTE', 'PERFEKT', '--', 'STEHST', 'ZEIGST', 'ECHT', 'LICHTER', 'STADT', 'VERBRANNT', 'LASS', 'STRAND', 'LASS', 'STRAND', 'WEISSEN', 'LASS', 'STRAND', 'YEAH', 'BRAUCHGRUND', 'BLEIBEN', 'AI', 'LASS', 'STRAND', 'WEISSEN', 'BRAUCH', 'MAL', 'WIEDER', 'MEER', 'SAND', 'YEAH', 'BRAUCH', 'MAL', 'WIEDERGRUND', 'BLEIBEN

In [9]:
print(df_Lied['Text'][0])

LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN ICH BRAUCH MAL WIEDER MEER UND SAND  YEAH BRAUCH MAL WIEDERGRUND ZU BLEIBEN  AI DEINE UHR FAELLT INS BLAU WIR SEHEN  WIE DIE ZEIT VERSINKT WIR SETZTEN LANGSAM AUF DAS WASSER SPIEGELGLATT UND BLINKT  WIR WOLLEN DIE SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS DU STEHST NEBEN MIR  ZEIGST MIR  DASS ES ECHT IST DIE LICHTER MEINER STADT SIND VERBRANNT LASS UNS ZUM STRAND    KOMM WIR STEIGEN AUS DU LEUCHTEST WIE AUS DIAMANT  YEHE MIT ALL DEM SALZ AUF DER HAUT  AI ICH SEH DICH GESTOCHEN SCHARF ES TUT FAST DEN AUGEN WEH UND ENDLICH BIST DU DA SO HABE ICH DICH NOCH NIE GESEHEN  YEHI  WIR WOLLEN DIE SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS DU STEHST NEBEN MIR  ZEIGST MIR  DASS ES ECHT IST DIE LICHTER MEINER STADT SIND VERBRANNT LASS UNS ZUM STRAND   LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN LASS UNS ZUM STRAND  YEAH ICH BRAUCHGRUND ZU BLEIB

Data Selection

In [57]:
print(df_Lied.head(20))

         ID        INTERPRET                                         TITEL  \
0   1148163            Yasha                                       Strand    
1   1148163            Yasha                                       Strand    
2   1148163            Yasha                                       Strand    
3   1148163            Yasha                                       Strand    
4   1148163            Yasha                                       Strand    
5   1148163            Yasha                                       Strand    
6   1148163            Yasha                                       Strand    
7   1148163            Yasha                                       Strand    
8   1148163            Yasha                                       Strand    
9   1148163            Yasha                                       Strand    
10  1148163            Yasha                                       Strand    
11     7341  Geier Sturzflug  Besuchen Sie Europa (solange es no

In [129]:
#df_Lied.drop(['TEXT_TEIL1','TEXT_TEIL2', 'TEXT_TEIL3', 'TEXT_TEIL4', 'SPRACHE_DEUTSCH'], axis=1, inplace=True)
#df_Date = df_Chart_Position[['LIED_ID', 'DATUM_VON', 'DATUM_BIS', 'DAUER','Jahr', 'Monat']]
#df_Chart_Position.drop(['DATUM_VON', 'DATUM_BIS', 'Jahr', 'Monat'], axis=1, inplace=True)
#df_Chart_Position = df_Chart_Position.groupby('LIED_ID').agg({'POSITION':'mean','DAUER':'sum'}).reset_index()
#df_Chart_Position['POSITION'] = df_Chart_Position.POSITION.apply(lambda pos: round(pos))
#df_Chart_Position = df_Chart_Position.rename({'LIED_ID':'ID'})
#print(df_Lied.columns)
#print(df_Chart_Position.columns)
#df = pd.merge(df_Lied, df_Chart_Position,left_on='ID', right_on='LIED_ID', how='inner')
print(df.head(30))

         ID        INTERPRET                                         TITEL  \
0   1148163            Yasha                                       Strand    
1   1148163            Yasha                                       Strand    
2   1148163            Yasha                                       Strand    
3   1148163            Yasha                                       Strand    
4   1148163            Yasha                                       Strand    
5   1148163            Yasha                                       Strand    
6   1148163            Yasha                                       Strand    
7   1148163            Yasha                                       Strand    
8   1148163            Yasha                                       Strand    
9   1148163            Yasha                                       Strand    
10  1148163            Yasha                                       Strand    
11     7341  Geier Sturzflug  Besuchen Sie Europa (solange es no

In [128]:
df['ANZ_UNIQUE_WOERTER'] = list(len(set(word)) for word in df['processed_Text'])
df['LAENGE_LAENGSTES_WORT'] = list(len(max(set(word), key=len)) for word in df['processed_Text'])

MAX_RANK = 50
df['RANK_SCORE'] = MAX_RANK - df['POSITION'] + 1

profanity check

In [116]:
with open(RAW_DATA_PATH+'german-profanity-list.csv', newline='', encoding='UTF-8') as f:
    profanity_list = list(csv.reader(f))

profanity.load_words(flatten(profanity_list))

Percentage of Stopwords

In [130]:
df['Number_of_Stopwords'] = df.Text.str.split().apply(lambda x: len(set(x) & set(stopwords_list)))
df['Stopword_Percentage'] = df.Number_of_Stopwords.apply(lambda row: round(row/len(df['Text']), ndigits=2))

Sentiment Analysis

In [131]:
df['SUBJEKTIVITY'] = df.processed_Text.apply(lambda text: TextBlob(str(text)).sentiment.subjectivity)
df['POLARITY'] = df.processed_Text.apply(lambda text: TextBlob(str(text)).sentiment.polarity)

In [134]:
df.drop(['LIED_ID'], axis=1, inplace=True)
print(df.head(20))

         ID        INTERPRET                                         TITEL  \
0   1148163            Yasha                                       Strand    
1   1148163            Yasha                                       Strand    
2   1148163            Yasha                                       Strand    
3   1148163            Yasha                                       Strand    
4   1148163            Yasha                                       Strand    
5   1148163            Yasha                                       Strand    
6   1148163            Yasha                                       Strand    
7   1148163            Yasha                                       Strand    
8   1148163            Yasha                                       Strand    
9   1148163            Yasha                                       Strand    
10  1148163            Yasha                                       Strand    
11     7341  Geier Sturzflug  Besuchen Sie Europa (solange es no

LDA (BoW) - Title

Named Entity Recognition

In [32]:
test = [word for text in df_Lied['Text'] for word in nlp(text).ents]
print(test[0])

KeyboardInterrupt: 

Readability analysis

Title Analysis

In [137]:
#titel_list = [word.upper() for word in list(set(df['TITEL']))]
#print(titel_list)
#lemmatize title
#df['processed_Title'] = df.TITEL.apply(lambda titel: ' '.join([x.lemma_ for x in nlp(titel)]))
#print(df['processed_Title'].head())
#tokenize title
tokenizer = RegexpTokenizer(r'\w+')
tokenized_list = df.processed_Title.apply(lambda titel: word for word in tokenizer.tokenize(titel))

#remove stopwords
no_stopwords_title_list = [item for item in tokenized_list if item not in stopwords_list]

#Analysis
#df_title_word_occurrence = pd.value_counts(np.array(no_stopwords_title_list))

['Strand', 'Strand', 'Strand', 'Strand', 'Strand', 'Strand', 'Strand', 'Strand', 'Strand', 'Strand', 'Strand', 'besuchen', 'sie', 'Europa', 'solange', 'es', 'noch', 'stehen', 'besuchen', 'sie', 'Europa', 'solange', 'es', 'noch', 'stehen', 'besuchen', 'sie', 'Europa', 'solange', 'es', 'noch', 'stehen', 'besuchen', 'sie', 'Europa', 'solange', 'es', 'noch', 'stehen', 'besuchen', 'sie', 'Europa', 'solange', 'es', 'noch', 'stehen', 'besuchen', 'sie', 'Europa', 'solange', 'es', 'noch', 'stehen', 'besuchen', 'sie', 'Europa', 'solange', 'es', 'noch', 'stehen', 'besuchen', 'sie', 'Europa', 'solange', 'es', 'noch', 'stehen', 'besuchen', 'sie', 'Europa', 'solange', 'es', 'noch', 'stehen', 'besuchen', 'sie', 'Europa', 'solange', 'es', 'noch', 'stehen', 'besuchen', 'sie', 'Europa', 'solange', 'es', 'noch', 'stehen', 'du', 'haben', 'Energie', 'Energie', 'Energie', 'Energie', 'Energie', 'Energie', 'an', 'der', 'Nordseeküste', 'an', 'der', 'Nordseeküste', 'an', 'der', 'Nordseeküste', 'an', 'der', 'Nor

Seasonal determination

Boosted Words -Analysis

Label Smoothing

Normalising

EDA

In [138]:
#create csv
#df_title_word_occurrence.to_csv('Data/processed/title_word_occurrence.csv')
#profanity Anteil

#sentiment Analyse
#Unique Wörter Anzahl Analyse

#Korreltaiton zwischen Interpret und Beliebtheit:
#df['INTERPRET'].value_counts(ascending=True, dropna=True, normalize=True)\
#.to_csv('Data/processed/Interpret_Occurrences_normalised.csv')

#df[['LAENGE_LAENGSTES_WORT', 'Jahr','ID','RANK_SCORE']].to_csv('Data/processed/Wort_LAENGE.csv')
#df[['ID','Number_of_Stopwords','Stopword_Percentage','Jahr', 'RANK_SCORE']].to_csv('Data/processed/Stoppwords.csv')

# Titel Analyse (länge des Titels, Wörter des Textes, ...)

df.to_csv('Data/processed/EDA.csv')